In [64]:
import torch
import numpy as np
temp = torch.rand(1, 2)
ft = torch.FloatTensor((1, 2))


In [65]:
ft

tensor([1., 2.])

In [67]:
temp_np = np.array(ft, dtype=np.uint8)
temp_np

array([1, 2], dtype=uint8)

In [53]:
temp_te = torch.from_numpy(temp_np)

In [62]:
temp_te = temp_te.to(torch.float32)

In [63]:
temp_te

tensor([[0., 0.]])